In [0]:
# Mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Importing required packages
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from skimage import data, filters

In [0]:
# Function to read image saved on google drive
def readImage(number, jpg = False):
    if jpg == True:
        path = "/content/drive/My Drive/images_ell715_assn5/" + str(number) + ".jpg"
    else:
        path = "/content/drive/My Drive/images_ell715_assn5/" + str(number) + ".tif"
    return cv.imread(path)

# Function to display image using matplotlib
def displayImage(a, title1 = "Img"):
    plt.imshow(a, cmap='gray'), plt.title(title1)
    plt.show()

In [0]:
# Area-based cleaning of image
def clean(img):
    # Find all connected components
    output = cv.connectedComponentsWithStats(np.uint8(img), 4, cv.CV_32S)
    num_labels = output[0]
    labels = output[1]
    stats = output[2]
    centroids = output[3]

    # output image
    cleaned = np.copy(np.uint8(img))
    # if area of any connected component is less than area threshold, remove that component
    areaThreshold = 25

    for i in range(cleaned.shape[0]):
        for j in range(cleaned.shape[1]):
            if stats[labels[i][j], cv.CC_STAT_AREA] < areaThreshold:
                cleaned[i][j] = 0
    return cleaned

In [0]:
# Extract green channel from image
def extractGreen(img):
    b, g, r = cv.split(img)
    displayImage(b, 'Blue Channel')
    displayImage(g, 'Green channel')
    displayImage(r, 'Red channel')
    return g

In [0]:
# Function to apply clahe on input image
def applyCLAHE(img):
    # hyper-parameters in clahe
    limit=3; grid=(7,7)
    clahe = cv.createCLAHE(clipLimit=limit, tileGridSize=grid)
    # output image
    img = clahe.apply(img)
    displayImage(img, "CLAHE")
    return img

In [0]:
# function to compute the gabor wavelets 
def gabor(r , c , kmax , f ,orient , scale , sig2 ): 
    gaborwavelet = np.zeros((r,c) , dtype = np.complex128)
    k = (kmax/(f ** scale)) * np.exp(1j * orient * (np.pi/8))
    kn2 = np.abs(k) ** 2

    for m in range(int(-r/2) + 1 , int(r/2) + 1):
        for n in range(int(-c/2) + 1 , int(c/2) + 1):
            temp1 = np.exp(1j * (np.real(k) * m + np.imag(k) * n ) ) - np.exp(-0.5 * sig2)
            temp2 = np.exp(-0.5 * kn2 * (m**2+n**2) / sig2 )
            gaborwavelet[int(m + r/2 - 1), int(n + c/2 -1)] = (kn2 / sig2) * temp2 * (temp1)
 
    return gaborwavelet

In [0]:
def applyGabor(img):
    # plotting gabor wavelets at different orientations
    fig = plt.figure()
    for x in range(0,8):
        gabor_wavelet = gabor(15, 15, np.pi / 2 , np.sqrt(2), x, 2, (np.pi/3) ** 2)
        fig.add_subplot(1,8,x+1)
        plt.imshow(np.real(gabor_wavelet) , cmap = 'gray')
    plt.title('Gabor')
    plt.show()

    final_img = np.zeros(img.shape)
    for u in range(0,8):
        gaussian_wavelet = gabor(15 , 15, np.pi / 2,np.sqrt(2) , u, 2, (np.pi/3) ** 2)
        t1 = cv.filter2D(img , cv.CV_32F, gaussian_wavelet.real) # taking convolution with real part to find the transformed image
        t2 = cv.filter2D(img , cv.CV_32F, gaussian_wavelet.imag) # taking convolution with imaginary part to find the transformed image
        temp = np.hypot(t1 , t2) # combining both output
        final_img = np.maximum(final_img , temp)
    return final_img

In [0]:
def wavelet(number):
    # read image
    img = readImage(number)
    # extract green channel
    img = extractGreen(img)
    # apply clahe on green channel
    img = applyCLAHE(img)
    # use gabor wavelets on clahe output
    final_img = applyGabor(img)

    #plotting final image
    plt.imshow(final_img , cmap = plt.get_cmap('gray'), vmin = 0 , vmax = np.max(final_img))
    plt.title('Image after applying gabor wavelets')
    plt.show()

    # applying hysteresis on image to highlight features
    hyst = filters.apply_hysteresis_threshold(final_img, 40, 40)
    # hysteresis on out
    displayImage(hyst, 'Hysteresis output')
    # area thresholding on hysteresis output image
    cleaned = clean(hyst)
    displayImage(cleaned, 'Cleaned')
    # closing operation to close holes in cleaned image
    closing = cv.morphologyEx(cleaned, cv.MORPH_CLOSE, kernel = np.ones((3,3),np.uint8))
    displayImage(closing, 'close')
    return closing

In [0]:
# Part 2
def sobel(img):
    # sobel operator on image
    scale = 1
    delta = 0
    ddepth = cv.CV_16S
    # 3*3 sobel operator
    grad_x = cv.Sobel(img, ddepth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv.BORDER_DEFAULT)
    grad_y = cv.Sobel(img, ddepth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv.BORDER_DEFAULT)
    abs_grad_x = cv.convertScaleAbs(grad_x)
    abs_grad_y = cv.convertScaleAbs(grad_y)
    grad = cv.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
    # display corresponding outputs from sobel operator
    displayImage(abs_grad_x,'sobel')
    displayImage(abs_grad_y,'sobel')
    displayImage(grad,'sobel')
    print(grad.max())
    print(grad.min())
    # hysteresis on sobel operator output image
    final_grad = filters.apply_hysteresis_threshold(grad,24,24)
    displayImage(final_grad,'binary')
    return final_grad

In [0]:
# Part 3
def LoG(img):
    # Applying gaussian on image
    blur = cv.GaussianBlur(img,(3,3),0)
    # Applying laplacian on the blur image to get the LOG output 
    LoG = cv.Laplacian(blur,cv.CV_16S)
    displayImage(LoG,'without_hyst_LOG')
    # Applying hystersis thersholding to convert the gray scale image to binary
    final_img = filters.apply_hysteresis_threshold(LoG,4,4)
    displayImage(final_img,'final_img_LOG')
    #final_img = clean(final_img)
    #displayImage(final_img,'cleaned_LoG')
    return final_img

In [0]:
# Function to run all three methods on image <number>.tif
def segment(number):
    wavelet(number)
    img = readImage(number)
    g = extractGreen(img)
    cv.imwrite("/content/drive/My Drive/images_ell715_assn5/"+str(number)+"_w.png",wavelet(number) * 255)   
    cv.imwrite("/content/drive/My Drive/images_ell715_assn5/"+str(number)+"_s.png", sobel(g) * 255)
    cv.imwrite("/content/drive/My Drive/images_ell715_assn5/"+str(number)+"_l.png", LoG(g) * 255) 

In [0]:
# Run code for all images
for i in range(1,21):
  segment(i)